In [ ]:
import math
import random as rd
import sys

def algo_Euclide(a,b):
    if(b==0):
        return a
    else:
        return algo_Euclide(b,a%b)
        
def exponentiation_modulaire (a, e, n):
    p=1
    while e>0:
        if (e % 2 != 0):
            p=(p*a)%n
            a=(a*a)%n
            e=e//2
    return p

def algo_Euclide_etendu(a,b):
    if(b==0):
        return(a,1,0)
    else:
        d,u,v=algo_Euclide_etendu(b, a%b)
        return d,v,u-(a//b)*v
        
def gen_pochon(n):
    tab=[]
    b=rd.randint(0,sys.maxsize//n)
    tab.append(b)
    n=n-1
    somme=b
    while (n>0):
        b=rd.randint(somme+1, somme + sys.maxsize//n)
        n=n-1
        somme+=b
        tab.append(b)
    return tab

def gen_W(M):
    W=rd.randint(1, M-1)
    while(math.gcd(W,M)!=1):
        W=rd.randint(1, M-1)
    return W

def gen_permutation(pochon,n):
    tab=[]
    while (len(tab)<n):
        i=rd.randint(0,n-1)
        if(tab.count(i)==0):
            tab.append(i)
    return tab

def gen_A(pochon,W,M,sigma,n):
    tabA=[]
    i=0
    while (len(tabA)<n):
        tabA.append((pochon[sigma[i]]*W)%M)
        i+=1
    return tabA

n=5
pochon=[2,5,11,23,55]
M=113
W=27
sigma=gen_permutation(pochon, n)
A=gen_A(pochon,W,M,sigma,n)
print(A)
print(sigma)    

## Génération clé privée, problème de pochon et déchiffrement

In [ ]:

from random import randint

"""
Génère une clé privée du cryptosystème de Kifli. Une clé privée contient un pochon P,
un entier M tel que la somme du pochon < M, W entier aléatoire entre 1 et M et premier avec ce dernier,
et sigma, représentant une permutation aléatoire des indices du pochon.

Paramètres:
    n : La taille du pochon et de sigma.

Retourne:
    (p, m, w, sigma) : La clé privée sous forme de n-uplet.
"""
def gen_cle_privee(n):
    # cle_privee = (P, M, W, sigma)
    # P = {b1, b2...}
    # M = M tel que somme bi < M
    # W = random entre 1 et M-1, premier avec M
    # sigma = permu aléatoire de P

    # Calcul de P 
    p = gen_pochon(n)

    # Calcul de M
    m = randint(sum(p) + 1, 10**64)

    # Calcul de W
    w = gen_W(m)

    # Calcul de sigma
    sigma = gen_permutation(p, n)
    
    return (p, m, w, sigma)

"""
Renvoie une solution au problème de pochon.

Paramètres :
    pochon : Le pochon dont on veut résoudre son problème.
    c : Le nombre à comparer avec les éléments du pochon.

Retour :
    E : La liste des indices du pochon qui résolvent le problème de pochon.
"""
def solve_pochon(pochon, c):
    res = 0 # La somme des éléments du pochon avant celui identifié
    E = [] # La liste des indices du pochon
    for i in range(0, len(pochon)):
        if P[i] + res <= c: # Ajout de 1 dans la liste si l'élément répond au problème
            E.append(1)
            res += pochon[i]
        else:
            E.append(0)
    return E

"""
Déchiffre un message chiffré par le cryptosystème de Kifli.

Paramètres:
    message_chiffre : Le message chiffré par le cryptosystème de Kifli. Il est
    représenté par une liste, qui sert de découpage du message.
    cle_privee : La clé privée utilisée.

Retourne :
    message_dechiffre : Le message déchiffré, sous forme de liste de listes. Chaque
    sous-liste est un bloc du message.
"""
def dechiffre(message_chiffre, cle_privee):
    # Inverse modulaire de W
    inverse_mod_w = algo_Euclide_etendu(cle_privee[2], cle_privee[1])[1] % cle_privee[1]
    
    # Nous avons considéré que la taille du dernier bloc de message devait être utilisée. Elle est le premier élément du message.
    taille_dernier_bloc = message_chiffre[0]

    message_dechiffre = []
    for message in message_chiffre:
        # Déterminer D tel que D = W^-1*message%M, avec W^-1 l'inverse modulaire de W.
        D = inverse_mod_w * message % cle_privee[1]
        # Résolution du problème de pochons
        P = cle_privee[0]
        P.reverse() # On utilise le pochon dans l'ordre décroissant
        E = solve_pochon(P, D)
        
        sous_message = ["0"]*len(cle_privee[3])
        for i in range(len(cle_privee[3])):
            sous_message[cle_privee[3][i]-1]=str(E[i]) # Ajout de l'indice de la permutation
        message_dechiffre.append(sous_message)
        
    # Découpage du dernier bloc, pour retirer les derniers éléments non-désirés, dans le cas où n est différent de la taille du message
    message_dechiffre[len(message_dechiffre)-1] = message_dechiffre[len(message_dechiffre)-1][:taille_dernier_bloc]
    return message_dechiffre
        
n = 5
P = [2,5,11,23,55]
M = 113
W = 27
sigma = (2,5,3,1,4)
priv=(P,M,W,sigma)
A = (22,16,71,54,56)
message="10101"
C = [4, 78, 22]

print(dechiffre(C, priv))


# Chiffrement d'un message 

In [ ]:
'''
Chiffre un message à partir d'une clé publique
d'un crypto-système de Kifli

Parametres :
    message      (str)   : Message encodé en binaire. ex : '10110'
    cle_publique (int[]) : Tableau d'entier correspondant à la clé publique

Retourne :
    int[] : Une liste de bloc de message crypté
'''
def chiffre(message, cle_publique):
    blocsMessageChiffre = []
    n = len(cle_publique) # n correspond à la taille maximum des 

    # On découpe tout d'abord le message en sous message d'une taille égale ou inférieure à la taille de la clé publique
    # Puis on parcours chacun de ces blocs
    for blocMessage in decoupMessage(message,n) :
        tailleDernierBloc = len(blocMessage)
        sommeC = 0
        for i in range (len(blocMessage)):
            if (message[i] == '1'):
                # On ajoute à la somme la valeur associé à la position du 1 dans le message qui est la valeur à l'index i du paquet A
                sommeC += cle_publique[i]
        
        
        blocsMessageChiffre.append((sommeC))

    return [tailleDernierBloc] + blocsMessageChiffre

'''
Découpe un message en plusieurs chaînes de caractères d'une taille n ou inférieure.
Seulement le dernier bloc peut avoir une taille inférieure.

Parametres :
    message (str) : Message à découper
    n       (int) : Taille 

Retourne :
    str[] : Une liste de blocs du message

'''
def decoupMessage(message,n):
    messageDecoupe = []

    #Découpage jusqu'à que le message initial soit vide
    while len(message) > 0 :
        messageDecoupe.append(message[:n]) # Récupèration d'un bloc de taille n
        message = message[n:] # Chagement du message afin d'enlever le bloc récupéré précedemment

    return messageDecoupe




cle_publique_test = [22,16,71,54,56]



assert(149 == chiffre('10101',cle_publique_test)[1])


print(chiffre('100011001',cle_publique_test))
print(chiffre('10001',cle_publique_test))



print(chiffre('1',cle_publique_test))
print(chiffre('10000',cle_publique_test))


print(chiffre('100011001',cle_publique_test))


## Test chiffrement déchiffrement

In [ ]:
def test_chiffrement_dechiffrement(message, n) :
    cle_privee = gen_cle_privee(n)
    cle_publique = gen_cle_publique(cle_privee)
    messageChiffre = chiffre(message, cle_publique)
    messageDechiffre = dechiffre(messageChiffre, cle_privee)
    return message == messageDechiffre

## D'après vous, le NIST a-t-il eu raison de ne pas retenir le crypto-système de Kifli dans sa liste d'algorithmes post-quantique ? Justifiez votre réponse.


N'ayant pas pu trouver d'informations complémentaire sur le crypto-système de Kifli donc à partir des informations donnés dans le sujet, il nous semble que ce crypto-système mérite une place dans la liste des algorithmes post-quantiques pour deux raisons.

En premier lieu, ce crypto-système semble respecter toutes les qualités que doivent respecter un crypto-système c'est à dire : 
- Confusion : Le message est intelligible lorsqu'il est chiffré.
- Diffusion : Une modification légère du message change le message suffisament pour ne pas déduire des informations.
- Robustesse de la clé : La robustesse de la clé dépend grandement de la capacité à générer aléatoirement une paire clé publique clé privée efficacement, de plus la longueur de la clé doit être assez grande pour qu'elle soit robuste. Cependant si elle est bien généré alors la clé est bien robuste.

En second lieu, il rompt avec le problème de factorisation que propose l'algorithme clé publique, clé privée utilisé en ce moment (RSA) qui est un problème qui sera résolu avec l'apparition des ordinateurs quantiques.
En proposant à la place un problème NP-complet qui n'ai pas d'algorithmes permettant de résoudre tous les cas en compléxité polynomial n.
De plus cet algorithme a peu de chance d'exister dans le futur car il n'est pas certain qu'il puisse exister un tel algorithme.


Cependant, il nous semble pertinent de mentionner qu'en tant que 2ème année de BUT Informatique nous connaissons pas assez la cryptographie et la cryptanalyse nous permettant d'avoir le recul nécessaire pour affirmer que cet algorithme soit viable ou non pour le futur.

